In [1]:
from theano.sandbox import cuda
#cuda.use('gpu2')
cuda.use('gpu0')

WARNING (theano.sandbox.cuda): The cuda backend is deprecated and will be removed in the next release (v0.10).  Please switch to the gpuarray backend. You can get more information about how to switch at this URL:
 https://github.com/Theano/Theano/wiki/Converting-to-the-new-gpu-back-end%28gpuarray%29

Using gpu device 0: GeForce GTX 950 (CNMeM is enabled with initial size: 90.0% of memory, cuDNN 5110)
WARNING (theano.sandbox.cuda): The cuda backend is deprecated and will be removed in the next release (v0.10).  Please switch to the gpuarray backend. You can get more information about how to switch at this URL:
 https://github.com/Theano/Theano/wiki/Converting-to-the-new-gpu-back-end%28gpuarray%29



In [2]:
%matplotlib inline
import utils; reload(utils)
from utils import *
from __future__ import division, print_function

Using Theano backend.


In [3]:
from keras.layers import TimeDistributed, Activation
from numpy.random import choice

## Setup

We haven't really looked into the detail of how this works yet - so this is provided for self-study for those who are interested. We'll look at it closely next week.

In [4]:
path = get_file('nietzsche.txt', origin="https://s3.amazonaws.com/text-datasets/nietzsche.txt")
text = open(path).read().lower()
print('corpus length:', len(text))

corpus length: 600901


In [5]:
!tail {path} -n25

are thinkers who believe in the saints.


144

It stands to reason that this sketch of the saint, made upon the model
of the whole species, can be confronted with many opposing sketches that
would create a more agreeable impression. There are certain exceptions
among the species who distinguish themselves either by especial
gentleness or especial humanity, and perhaps by the strength of their
own personality. Others are in the highest degree fascinating because
certain of their delusions shed a particular glow over their whole
being, as is the case with the founder of christianity who took himself
for the only begotten son of God and hence felt himself sinless; so that
through his imagination--that should not be too harshly judged since the
whole of antiquity swarmed with sons of god--he attained the same goal,
the sense of complete sinlessness, complete irresponsibility, that can
now be attained by every individual through science.--In the same manner
I have viewed the saints of India

In [6]:
#path = 'data/wiki/'
#text = open(path+'small.txt').read().lower()
#print('corpus length:', len(text))

#text = text[0:1000000]

In [7]:
chars = sorted(list(set(text)))
vocab_size = len(chars)+1
print('total chars:', vocab_size)

total chars: 60


In [8]:
chars

['\n',
 ' ',
 '!',
 '"',
 "'",
 '(',
 ')',
 ',',
 '-',
 '.',
 '0',
 '1',
 '2',
 '3',
 '4',
 '5',
 '6',
 '7',
 '8',
 '9',
 ':',
 ';',
 '=',
 '?',
 '[',
 ']',
 '_',
 'a',
 'b',
 'c',
 'd',
 'e',
 'f',
 'g',
 'h',
 'i',
 'j',
 'k',
 'l',
 'm',
 'n',
 'o',
 'p',
 'q',
 'r',
 's',
 't',
 'u',
 'v',
 'w',
 'x',
 'y',
 'z',
 '\x86',
 '\xa4',
 '\xa6',
 '\xa9',
 '\xab',
 '\xc3']

In [9]:
chars.insert(0, "\0")

In [10]:
''.join(chars[1:-6])

'\n !"\'(),-.0123456789:;=?[]_abcdefghijklmnopqrstuvwxyz'

In [11]:
char_indices = dict((c, i) for i, c in enumerate(chars))
indices_char = dict((i, c) for i, c in enumerate(chars))

In [12]:
char_indices

{'\x00': 0,
 '\n': 1,
 ' ': 2,
 '!': 3,
 '"': 4,
 "'": 5,
 '(': 6,
 ')': 7,
 ',': 8,
 '-': 9,
 '.': 10,
 '0': 11,
 '1': 12,
 '2': 13,
 '3': 14,
 '4': 15,
 '5': 16,
 '6': 17,
 '7': 18,
 '8': 19,
 '9': 20,
 ':': 21,
 ';': 22,
 '=': 23,
 '?': 24,
 '[': 25,
 ']': 26,
 '_': 27,
 'a': 28,
 'b': 29,
 'c': 30,
 'd': 31,
 'e': 32,
 'f': 33,
 'g': 34,
 'h': 35,
 'i': 36,
 'j': 37,
 'k': 38,
 'l': 39,
 'm': 40,
 'n': 41,
 'o': 42,
 'p': 43,
 'q': 44,
 'r': 45,
 's': 46,
 't': 47,
 'u': 48,
 'v': 49,
 'w': 50,
 'x': 51,
 'y': 52,
 'z': 53,
 '\x86': 54,
 '\xa4': 55,
 '\xa6': 56,
 '\xa9': 57,
 '\xab': 58,
 '\xc3': 59}

In [13]:
indices_char

{0: '\x00',
 1: '\n',
 2: ' ',
 3: '!',
 4: '"',
 5: "'",
 6: '(',
 7: ')',
 8: ',',
 9: '-',
 10: '.',
 11: '0',
 12: '1',
 13: '2',
 14: '3',
 15: '4',
 16: '5',
 17: '6',
 18: '7',
 19: '8',
 20: '9',
 21: ':',
 22: ';',
 23: '=',
 24: '?',
 25: '[',
 26: ']',
 27: '_',
 28: 'a',
 29: 'b',
 30: 'c',
 31: 'd',
 32: 'e',
 33: 'f',
 34: 'g',
 35: 'h',
 36: 'i',
 37: 'j',
 38: 'k',
 39: 'l',
 40: 'm',
 41: 'n',
 42: 'o',
 43: 'p',
 44: 'q',
 45: 'r',
 46: 's',
 47: 't',
 48: 'u',
 49: 'v',
 50: 'w',
 51: 'x',
 52: 'y',
 53: 'z',
 54: '\x86',
 55: '\xa4',
 56: '\xa6',
 57: '\xa9',
 58: '\xab',
 59: '\xc3'}

In [14]:
idx = [char_indices[c] for c in text]

In [15]:
idx[:10]

[43, 45, 32, 33, 28, 30, 32, 1, 1, 1]

In [16]:
''.join(indices_char[i] for i in idx[:70])

'preface\n\n\nsupposing that truth is a woman--what then? is there not gro'

## Preprocess and create model

In [18]:
maxlen = 40 # time step in another word
sentences = []
next_chars = []
for i in range(0, len(idx) - maxlen+1): # 600901 - 40 + 1 = 600862
    sentences.append(idx[i: i + maxlen]) # list in list
    next_chars.append(idx[i+1: i+maxlen+1])
print('nb sequences:', len(sentences))

nb sequences: 600862


In [18]:
sentences = np.concatenate([[np.array(o)] for o in sentences[:-2]]) # why -2 ??
next_chars = np.concatenate([[np.array(o)] for o in next_chars[:-2]])

In [19]:
sentences.shape, next_chars.shape

((600860, 40), (600860, 40))

In [20]:
n_fac = 24

In [21]:
# return_sequences: Boolean. Whether to return the last output in the output sequence, or the full sequence.

# TimeDistributed
#    https://machinelearningmastery.com/timedistributed-layer-for-long-short-term-memory-networks-in-python/
#    https://www.quora.com/What-is-time-distributed-dense-layer-in-Keras
#    https://datascience.stackexchange.com/questions/10836/the-difference-between-dense-and-timedistributeddense-of-keras

model=Sequential([
        Embedding(vocab_size, n_fac, input_length=maxlen),
        LSTM(512, input_dim=n_fac,return_sequences=True, dropout_U=0.2, dropout_W=0.2,
             consume_less='gpu'),
        Dropout(0.2),
        LSTM(512, return_sequences=True, dropout_U=0.2, dropout_W=0.2,
             consume_less='gpu'),
        Dropout(0.2),
        TimeDistributed(Dense(vocab_size)), # apply a Dense layer on each timestep
        Activation('softmax')
    ])    

In [22]:
model.compile(loss='sparse_categorical_crossentropy', optimizer=Adam())

In [24]:
model.summary()

____________________________________________________________________________________________________
Layer (type)                     Output Shape          Param #     Connected to                     
embedding_1 (Embedding)          (None, 40, 24)        1440        embedding_input_1[0][0]          
____________________________________________________________________________________________________
lstm_1 (LSTM)                    (None, 40, 512)       1099776     embedding_1[0][0]                
____________________________________________________________________________________________________
dropout_1 (Dropout)              (None, 40, 512)       0           lstm_1[0][0]                     
____________________________________________________________________________________________________
lstm_2 (LSTM)                    (None, 40, 512)       2099200     dropout_1[0][0]                  
___________________________________________________________________________________________

## Train

In [29]:
def print_example():
    seed_string="ethics is a basic foundation of all that"
    for i in range(320):
        x=np.array([char_indices[c] for c in seed_string[-40:]])[np.newaxis,:] # shape is (1, 40)    
        # model.predict(x) returns (1, 40, 60)
        #   40: input sequence size (time steps)
        #   60: vocab_size
        preds = model.predict(x, verbose=0)[0][-1] # (60,), prediction for last character in input sequence
        preds = preds/np.sum(preds)
        next_char = choice(chars, p=preds)
        seed_string = seed_string + next_char
    print(seed_string)

In [30]:
print(sentences.shape)
print(np.expand_dims(next_chars, -1).shape)

(600860, 40)
(600860, 40, 1)


In [31]:
# We have to reshape from (40,) to (40,1) for label data to use TimeDistributed
# Each step in sequence have a label (1,)

# sentences: (600860, 40)
# np.expand_dims(next_chars, -1): (600860, 40, 1)
model.fit(sentences, np.expand_dims(next_chars,-1), batch_size=64, nb_epoch=1) 

Epoch 1/1
600860/600860 [==============================] - 1313s - loss: 1.5202  


In [32]:
print_example()

ethics is a basic foundation of all that it
is betracted to our eximinating misings.--as play,
since
into the health, and,
indeed, now dangered upon what something
was precisely its brain by the modes of their
religious but has been exertion to europe we says we have no
existence and philosophers, in its first
of which the excitation of taste,
but sare speci


In [33]:
model.fit(sentences, np.expand_dims(next_chars,-1), batch_size=64, nb_epoch=1)

Epoch 1/1
600860/600860 [==============================] - 1313s - loss: 1.2937  


In [34]:
print_example()

ethics is a basic foundation of all that
evil was higher development of the
lap of the struggle of the
best of himself most
refinement
and absolutely even sare nature. because they
would have seens shape
to longing which, however, of which it except as if rather said to the sense of them as the body: yet happiness,
longing, religion, costumes, of rank so wer


In [35]:
model.optimizer.lr=0.001

In [36]:
model.fit(sentences, np.expand_dims(next_chars,-1), batch_size=64, nb_epoch=1)

Epoch 1/1
600860/600860 [==============================] - 1315s - loss: 1.2572  


In [37]:
print_example()

ethics is a basic foundation of all that should be
reflector of them who takes place
comprehensive animal, the fact that there are empty on what woman of
things are always further
doly in the fact that the part of life. and of the supposition of
the most succeed, and lapse of conjuring absolute state of
justificated class valuations of your own late, given n


In [38]:
model.optimizer.lr=0.0001

In [39]:
model.fit(sentences, np.expand_dims(next_chars,-1), batch_size=64, nb_epoch=1)

Epoch 1/1
600860/600860 [==============================] - 1313s - loss: 1.2371  


In [40]:
print_example()

ethics is a basic foundation of all that woman is blunded.

80. woman there is no longer--     by a kind of guiltlessness for christian while has more
more to the appearance of the
arts of those things learn to)
and respossible for an inversion are compared by progress. if any will,
and without praise, and explained, if it were, to
remain,
these very final b


In [41]:
model.save_weights('data/char_rnn.h5')

In [42]:
model.optimizer.lr=0.00001

In [43]:
model.fit(sentences, np.expand_dims(next_chars,-1), batch_size=64, nb_epoch=1)

Epoch 1/1
600860/600860 [==============================] - 1315s - loss: 1.2226  


In [44]:
print_example()

ethics is a basic foundation of all that
philosophy
can display one, and perhaps not always
lies "according here," and "staging his state in
such
heart," or fears and powers and temps,
     a child is the origin of done with a physiological minds
to us. and what we
have changed as to the sensation with regard to arguments, there was
to see the berangpo who h


In [45]:
model.fit(sentences, np.expand_dims(next_chars,-1), batch_size=64, nb_epoch=1)

Epoch 1/1
600860/600860 [==============================] - 1314s - loss: 1.2101  


In [46]:
print_example()

ethics is a basic foundation of all that is standing rest
strong testing by explanations was not that the psychological kind of thought that the athomation of the intellectual and imitating form of the toco that they feel ourselves with cognizing and purity and more flashed, the injury in that this is often called classification of
the ears which can be tran


In [47]:
print_example()

ethics is a basic foundation of all that strangely came for it. one has clination
on such admiration, so seems that they evolved in his
immense, even if
one does
not more fixmative and
destroying) of the greek and cases, who believes
if sensuousness, something that in my
blood--damely, in the good acted confront that of
the type of depth of morally, is an in


In [48]:
model.save_weights('data/char_rnn.h5')